![DLI Header](images/DLI_Header.png)

# 迭代式提示词开发

在本笔记本中，我们将通过迭代一系列简单的提示词来进行热身，熟悉我们将在整个课程中使用的 `transformers` 管道和 LLaMA-2 模型。

通过迭代式地实验看似简单的提示词，我们将开始看到创建**具体**的、提供**线索**的提示词的重要性，还将学习如何在给模型提供它可能觉得具有挑战性的任务时，给模型**“时间思考”**。

## 学习目标

当你完成本笔记本时，你将能够：
- 使用 `transformers` 管道从 LLaMA-2 大型语言模型生成响应。
- 制作**具体**的提示词。
- 制作给模型**“时间思考”**的提示词。
- 为模型提供**线索**以引导其响应。

## 视频演练

执行下面的单元格以加载本笔记本的视频演练。 

In [5]:
from IPython.display import HTML

# 视频地址
video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/02-prompting.mp4"

# 用于嵌入视频的 HTML 代码
video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    您的浏览器不支持 video 标签。 
</video>
"""

# 显示视频
display(HTML(video_html))

## 创建 LLaMA-2 管道

In [7]:
# 导入 pipeline 模块
from transformers import pipeline

# 指定要使用的模型
model = "TheBloke/Llama-2-13B-chat-GPTQ"

# 创建文本生成管道，并指定模型和设备
llama_pipe = pipeline("text-generation", model=model, device_map="auto"); 

## 辅助函数

在本笔记本中，我们将使用以下函数来支持我们与大型语言模型的交互。

### 生成模型响应

In [8]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    使用指定的语言模型管道生成对给定提示的响应。

    此函数接收一个提示并将其传递给语言模型管道（例如 LLaMA）以生成文本响应。
    该函数旨在允许通过各种参数和关键字参数自定义生成过程。

    参数：
    - prompt (str): 用于生成响应的输入文本提示。
    - max_length (int): 生成的响应的最大长度。默认为 1024 个标记。
    - pipe (callable): 用于生成的语言模型管道函数。默认为 llama_pipe。
    - ** kwargs: 传递给管道函数的其他关键字参数。

    返回值：
    - str: 模型生成的文本响应，已去除前导和尾随空格。

    使用示例：
    ```
    prompt_text = "解释相对论。"
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, ** def_kwargs)
    return response[0]['generated_text'].strip()

## 加利福尼亚州的首府

让我们从一个非常简单的提示词开始，我们将把它传递给我们的 `generate` 函数，以便从我们正在使用的 LLaMA-2 模型获得响应。在本系列提示词中，我们希望模型用加利福尼亚州的首府 *Sacramento* 来回答我们。

我们希望这次实验能够得到准确的响应 `"Sacramento"`，而不需要任何其他内容。

In [9]:
# 设置提示词，询问加利福尼亚州的首府
prompt = "加利福尼亚州的首府是什么？"  

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt)) 

A. 旧金山
B. 洛杉矶
C. 萨克拉门托
D. 法尔的贝尼托

答案：C. 萨克拉门托

解释：加利福尼亚州的首府是萨克拉门托（Sacramento），而不是旧金山（San Francisco）、洛杉矶（Los Angeles）或法尔的贝尼托（Fresno）。


---

该模型不明白我们只想要首府城市的名称，而不需要任何其他上下文，所以让我们制作一个更**具体**的提示词。

In [27]:
# 设置提示词，要求模型只回答加利福尼亚州的首府，并尽可能简洁
prompt = "加利福尼亚州的首府的英文名称是什么？请只回答这个问题，并尽可能用最少的词。" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

Answer: Sacramento.


---

这是一个改进，但我们在响应中仍然得到了一个前导的 `Answer: `。让我们尝试通过给模型提供**线索** `Answer: ` 来防止这种行为。这样做可能会阻止模型自己提供该文本。

In [12]:
# 设置提示词，要求模型只回答加利福尼亚州的首府，并尽可能简洁，同时提供 "Answer: " 作为引导
prompt = "加利福尼亚州的首府是什么？请只回答这个问题，并尽可能用最少的词。Answer: " 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

Sacramento。


## Sacramento 中的元音

在下一节中，我们将尝试让模型做一些更复杂的事情：告诉我们加利福尼亚州首府名称中的所有元音。

正确答案是 S**a**cr**a**m**e**nt**o** -> **aaeo** -> **aeo**。值得注意的是，为了方便我自己（和你），我执行了多个步骤才得到我的答案。

In [16]:
# 设置提示词，询问加利福尼亚州首府名称中的元音字母
prompt = "告诉我加利福尼亚州首府名称（英文）中的元音字母。"

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

加利福尼亚州首府名称（英文）中的元音字母有哪些？

加利福尼亚州首府名称（英文）中的元音字母有以下几个：

1. A (在 "Sacramento" 中)
2. E (在 "Sacramento" 中)
3. I (在 "Sacramento" 中)

因此，加利福尼亚州首府名称（英文）中的元音字母有 A, E, I 三个。


---

当模型面临需要多步骤推理的情况时，构建一个请求模型执行多个中间步骤的提示通常会有所帮助，就像要求它显示其工作原理一样。这种技术通常被称为给模型**“时间思考”**。

下面的提示旨在获得相同的最终结果，但要求模型先回答加利福尼亚州的首府，然后再回答其中的元音。

In [17]:
# 设置提示词，先询问加利福尼亚州的首府，然后询问其中的元音字母
prompt = "告诉我加利福尼亚州的首府（英文），然后告诉我其中的所有元音字母。" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

首府是 Sacramento，其中的元音字母有：

* a (as in "cat")
* e (as in "bet")
* o (as in "go")

所以，加利福尼亚州的首府 Sacramento 的元音字母有 a、e、o 三个。


---

现在我们看到了给模型**“时间思考”**的有效性，让我们尝试一个稍微复杂一点的任务：按字母倒序排列加利福尼亚州首府中的元音。

正确答案是 S**a**cr**a**m**e**nt**o** -> **aaeo** -> **aeo** -> **oea**

In [18]:
# 设置提示词，询问加利福尼亚州首府名称中的元音字母，并按字母倒序排列
prompt = "告诉我加利福尼亚州首府（英文）名称中的元音字母，并按字母倒序排列？"

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

首先，加利福尼亚州首府的英文名称是 Sacramento。

Sacramento 中的元音字母有：

* a (在 "a" 的位置)
* e (在 "e" 的位置)
* o (在 "o" 的位置)

按字母倒序排列，我们得到：

* o
* e
* a


---

为了帮助模型，让我们再次给它**“时间思考”**，提示它将任务分解成中间步骤并显示其工作原理。

In [28]:
# 设置提示词，先询问加利福尼亚州的首府，然后询问其中的元音字母，最后按字母倒序排列这些元音字母
prompt = "告诉我加利福尼亚州的首府的英文名称，然后告诉我这个英文名称中的所有元音字母，最后按字母倒序排列这些元音字母。"

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

请问：

1. 加利福尼亚州的首府的英文名称是什么？
2. 这个英文名称中的所有元音字母是什么？
3. 按字母倒序排列这些元音字母。

请问1，加利福尼亚州的首府的英文名称是什么？

Answer 1: The capital of California is Sacramento.

请问2, 这个英文名称中的所有元音字母是什么？

Answer 2: The vowels in "Sacramento" are "a", "e", and "o".

请问3, 按字母倒序排列这些元音字母。

Answer 3: The vowels in "Sacramento" in alphabetical order are "e", "a", and "o".


## 练习

虽然大型语言模型不一定是执行数学运算的最佳工具，但作为练习，请从下面的提示生成响应，该提示旨在获得 23 和 34 相乘的积，然后迭代开发一个提示，使您得到正确答案。请务必考虑如何在提示中做到**精确**，并为模型提供**“时间思考”**的机会。

如果您遇到困难，下面提供了一个解决方案。

In [34]:
23*34  # 显示实际答案

782

In [35]:
# 设置提示词，这个提示词对我们来说很清楚，但对模型来说不够精确
prompt = "23x34" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

cm.

### Versions

There are several versions of the painting, including:

1. The original version, which is considered the most authentic and is housed in the Louvre Museum in Paris.
2. A version from the early 16th century, which is housed in the National Gallery in London.
3. A version from the late 16th century, which is housed in the Prado Museum in Madrid.
4. A version from the 17th century, which is housed in the Hermitage Museum in St. Petersburg.

Each version has its own unique characteristics and details, but they all share the same overall composition and theme.

### Symbolism

The painting is rich in symbolism, with many objects and figures representing different aspects of life and death. Some of the most notable symbols include:

1. The Fates: The three women in the painting, who represent the Fates, or the goddesses of destiny. They are depicted as spinning the thread of life, which represents the cycle of life and death.
2. The Angel: The angel in the painting, who is 

### 你的工作区

### 解决方案

点击 `...` 查看可行的解决方案。

In [42]:
# 设置提示词，要求模型计算 23 和 34 的乘积，并使用长乘法步骤展示计算过程
prompt = "计算 23 与 34 的乘积。并使用长乘法步骤（the steps typical of long multiplication）展示计算过程" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

。

计算结果应该是：

23 × 34 = 761

以下是长乘法步骤：

1. 将 23 与 34 分别写入两个数字矩阵中：

| 23 |
| --- |
| 34 |

2. 计算每个数字的乘积，从左到右，从上到下：

| 23 | x | 34 | = | 23 x 34 |

3. 将每个乘积写入对应的数字矩阵中：

| 23 | x | 34 | = | 761 |

4. 将所有乘积写入一个矩阵中，以便更方便地计算：

| 23 | x | 34 | = | 761 |

5. 计算矩阵的和，即：

| 23 | x | 34 | = | 761 |

6. 最后，将矩阵的和作为计算结果：

23 × 34 = 761

这个过程可以帮助你更好地理解和记忆长乘法的步骤。


## 关键概念回顾

本笔记本介绍了以下关键概念：

- **精确**：尽可能明确地引导大型语言模型的响应。
- **线索**：提示的结尾，用于引导其响应，通常是为了防止其在响应中包含线索本身。
- **“时间思考”**：提示中的一种质量，通过要求模型采取多个步骤并显示其工作原理来支持大型语言模型的响应（通常需要计算）。

## 重启内核

|Restart the Kernel

为了给下一个笔记本释放 GPU 内存，请运行以下单元格以重启内核。

In [43]:
# 导入 get_ipython 函数
from IPython import get_ipython 

# 获取当前 IPython 内核并执行重启操作
get_ipython().kernel.do_shutdown(restart=True) 

{'status': 'ok', 'restart': True}

# [5LOI DEEP LEARNING INSTITUE](https://5loi.com)

### [AI COMMUNITY](https://www.theforage.cn/community)

### [5LOI](https://5loi.com/about_loi)

![DLI Header](images/DLI_Header.png)